In [0]:
import keras
import numpy as np
from keras.applications import resnet50
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

resnet_model = resnet50.ResNet50(weights='imagenet')

Using TensorFlow backend.


102858752/102853048 [==============================] - 1s 0us/step


In [0]:
import glob
import numpy as np
import pandas as pd
import os
import shutil 
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [0]:
IMG_WIDTH=300
IMG_HEIGHT=300
IMG_DIM = (IMG_WIDTH, IMG_HEIGHT)

validation_files = glob.glob('/content/drive/My Drive/ML Project/Brian/pets/test/*')
validation_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in validation_files]
validation_imgs = np.array(validation_imgs)
validation_labels = [fn.split('/')[-1].split('.')[0].strip() for fn in validation_files]

train_files = glob.glob('/content/drive/My Drive/ML Project/Brian/pets/train/*') 
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)
train_labels = [fn.split('/')[-1].split('.')[0].strip() for fn in train_files]


print('Train dataset shape:', train_imgs.shape, 
 '\tValidation dataset shape:', validation_imgs.shape)

Train dataset shape: (5900, 300, 300, 3) 	Validation dataset shape: (1463, 300, 300, 3)


In [0]:
train_imgs_scaled = train_imgs.astype('float32') 
validation_imgs_scaled = validation_imgs.astype('float32') 
train_imgs_scaled /= 255 
validation_imgs_scaled /= 255 

In [0]:
# encode text category labels 
from sklearn.preprocessing import LabelEncoder 
 
le = LabelEncoder() 
le.fit(train_labels) 
train_labels_enc = le.transform(train_labels) 
validation_labels_enc = le.transform(validation_labels) 
 


NUM_CLASSES = 37
train_labels = keras.utils.to_categorical(train_labels_enc, NUM_CLASSES)
validation_labels = keras.utils.to_categorical(validation_labels_enc, NUM_CLASSES)
#print(train_labels[60:70], train_labels_enc[60:70])

ValueError: ignored

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
 width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
 horizontal_flip=True, fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow(train_imgs, train_labels,batch_size=30)
val_generator = val_datagen.flow(validation_imgs, validation_labels, batch_size=30)

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.applications import vgg16
from keras.models import Model
import keras
VGG16_MODEL = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3))
output = VGG16_MODEL.layers[-1].output
output = keras.layers.Flatten()(output)
VGG16_MODEL = Model(VGG16_MODEL.input, output=output)
for layer in VGG16_MODEL.layers:
    layer.trainable = False
#VGG16_MODEL.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("fl...)`
  


In [0]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model = Sequential()
model.add(VGG16_MODEL)
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(37, activation='softmax'))

opt=optimizers.RMSprop(lr=2e-5)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 41472)             14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 512)               21234176  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 37)                18981     
Total params: 36,230,501
Trainable params: 21,515,813
Non-trainable params: 14,714,688
_________________________________

In [0]:
#history = model.fit_generator(train_generator, 
#                              steps_per_epoch=20, 
#                              epochs=20,
#                              validation_data=val_generator, 
#                              validation_steps=10, 
#                              verbose=1)


history = model.fit(train_imgs_scaled, train_labels,
                    epochs=20, 
                    batch_size=128,
                    validation_data=(validation_imgs_scaled, validation_labels),
                    shuffle=True,verbose=1)

Train on 5900 samples, validate on 1463 samples
Epoch 1/20
5900/5900 [==============================] - 34s 6ms/step - loss: 2.7762 - accuracy: 0.1595 - val_loss: 2.3128 - val_accuracy: 0.2850
Epoch 2/20
5900/5900 [==============================] - 34s 6ms/step - loss: 2.4327 - accuracy: 0.2312 - val_loss: 2.0147 - val_accuracy: 0.3718
Epoch 3/20
5900/5900 [==============================] - 34s 6ms/step - loss: 2.1614 - accuracy: 0.3005 - val_loss: 1.9504 - val_accuracy: 0.3794
Epoch 4/20
5900/5900 [==============================] - 34s 6ms/step - loss: 1.9663 - accuracy: 0.3536 - val_loss: 1.8260 - val_accuracy: 0.4272
Epoch 5/20
5900/5900 [==============================] - 34s 6ms/step - loss: 1.8018 - accuracy: 0.4034 - val_loss: 1.8036 - val_accuracy: 0.4238
Epoch 6/20
5900/5900 [==============================] - 34s 6ms/step - loss: 1.6185 - accuracy: 0.4659 - val_loss: 1.7446 - val_accuracy: 0.4511
Epoch 7/20
5900/5900 [==============================] - 34s 6ms/step - loss: 1.500

In [0]:
print(train_imgs_scaled.shape)
print(train_labels.shape)
print(validation_imgs_scaled.shape)
print(validation_labels.shape)